In [ ]:
# import training and testing dataset
import pandas as pd
destinations = pd.read_csv("destinations.csv")
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [ ]:
#assign unique Id and generate data feature for test and training dataset
test_ids = set(test.user_id.unique())
train_ids = set(train.user_id.unique())
intersection_count = len(test_ids & train_ids)
intersection_count == len(test_ids)
train["date_time"] = pd.to_datetime(train["date_time"])
train["year"] = train["date_time"].dt.year
train["month"] = train["date_time"].dt.month

In [ ]:
#select 10000 data for our experiment
import random

unique_users = train.user_id.unique()

sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 10000)) ]
sel_train = train[train.user_id.isin(sel_user_ids)]

In [ ]:
t1 = sel_train[((sel_train.year == 2013) | ((sel_train.year == 2014) & (sel_train.month < 8)))]
t2 = sel_train[((sel_train.year == 2014) & (sel_train.month >= 8))]
t2 = t2[t2.is_booking == True]

In [ ]:
#predict with common top-5 clusters
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)
predictions = [most_common_clusters for i in range(t2.shape[0])]
#calculate MAP@5 score
import ml_metrics as metrics
target = [[l] for l in t2["hotel_cluster"]]
metrics.mapk(target, predictions, k=5)

In [ ]:
train.corr()["hotel_cluster"]

In [ ]:
#use PCA to convert 149 features to 3
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
dest_small = pca.fit_transform(destinations[["d{0}".format(i + 1) for i in range(149)]])
dest_small = pd.DataFrame(dest_small)
dest_small["srch_destination_id"] = destinations["srch_destination_id"]

In [ ]:
#generate new features
def generate_new_feature(df):
    df["date_time"] = pd.to_datetime(df["date_time"])
    df["srch_ci"] = pd.to_datetime(df["srch_ci"], format='%Y-%m-%d', errors="coerce")
    df["srch_co"] = pd.to_datetime(df["srch_co"], format='%Y-%m-%d', errors="coerce")
    
    props = {}
    for prop in ["month", "day", "hour", "minute", "dayofweek", "quarter"]:
        props[prop] = getattr(df["date_time"].dt, prop)
    
    carryover = [p for p in df.columns if p not in ["date_time", "srch_ci", "srch_co"]]
    for prop in carryover:
        props[prop] = df[prop]
    
    date_props = ["month", "day", "dayofweek", "quarter"]
    for prop in date_props:
        props["ci_{0}".format(prop)] = getattr(df["srch_ci"].dt, prop)
        props["co_{0}".format(prop)] = getattr(df["srch_co"].dt, prop)
    props["stay_span"] = (df["srch_co"] - df["srch_ci"]).astype('timedelta64[h]')
        
    ret = pd.DataFrame(props)
    
    ret = ret.join(dest_small, on="srch_destination_id", how='left', rsuffix="dest")
    ret = ret.drop("srch_destination_iddest", axis=1)

    return ret

df = generate_new_feature(t1)
df.fillna(-1, inplace=True)

In [ ]:
predictors = [c for c in df.columns if c not in ["hotel_cluster"]]
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier
#Random decision Forest
clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.05)
scores = cross_validation.cross_val_score(clf, df[predictors], df['hotel_cluster'], cv=3)
scores

In [77]:
#binary classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from itertools import chain

all_probs = []
unique_clusters = df["hotel_cluster"].unique()
for cluster in unique_clusters:
    df["target"] = 1
    df["target"][df["hotel_cluster"] != cluster] = 0
    predictors = [col for col in df if col not in ['hotel_cluster', "target"]]
    probs = []
    cv = KFold(len(df["target"]), n_folds=2)
    clf = RandomForestClassifier(n_estimators=10, min_weight_fraction_leaf=0.1)
    for i, (tr, te) in enumerate(cv):
        clf.fit(df[predictors].iloc[tr], df["target"].iloc[tr])
        preds = clf.predict_proba(df[predictors].iloc[te])
        probs.append([p[1] for p in preds])
    full_probs = chain.from_iterable(probs)
    all_probs.append(list(full_probs))

prediction_frame = pd.DataFrame(all_probs).T
prediction_frame.columns = unique_clusters
def find_top_5(row):
    return list(row.nlargest(5).index)

preds_m = []
for index, row in prediction_frame.iterrows():
    preds_m.append(find_top_5(row))

metrics.mapk([[l] for l in t2["hotel_cluster"]], preds_m, k=5)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.044268287837247192

In [78]:
metrics.mapk([[l] for l in t2["hotel_cluster"]], preds_m, k=5)

0.044268287837247192

In [79]:
def make_key(items):
    return "_".join([str(i) for i in items])

match_cols = ["srch_destination_id"]
cluster_cols = match_cols + ['hotel_cluster']
groups = t1.groupby(cluster_cols)
top_clusters = {}
for name, group in groups:
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True])
    
    score = bookings + .3 * clicks
    
    clus_name = make_key(name[:len(match_cols)])
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {}
    top_clusters[clus_name][name[-1]] = score

In [81]:
import operator

cluster_dict = {}
for n in top_clusters:
    tc = top_clusters[n]
    top = [l[0] for l in sorted(tc.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    cluster_dict[n] = top

In [83]:
preds = []
for index, row in t2.iterrows():
    key = make_key([row[m] for m in match_cols])
    if key in cluster_dict:
        preds.append(cluster_dict[key])
    else:
        preds.append([])

In [85]:
metrics.mapk([[l] for l in t2["hotel_cluster"]], preds, k=5)

0.2373425517542814

In [86]:
match_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance']

groups = t1.groupby(match_cols)
    
def generate_exact_matches(row, match_cols):
    index = tuple([row[t] for t in match_cols])
    try:
        group = groups.get_group(index)
    except Exception:
        return []
    clus = list(set(group.hotel_cluster))
    return clus

exact_matches = []
for i in range(t2.shape[0]):
    exact_matches.append(generate_exact_matches(t2.iloc[i], match_cols))

In [103]:
def weigh(seq, weight):
    dict = {}
    for key in seq:
        dict[key] = weight
    return dict

def merge(dicts):
    result = []
    mergeDict = {}
    for dict in dicts:
        for key in dict.keys():
            if key in mergeDict.keys():
                mergeDict[key] += dict[key]
            else:
                mergeDict[key] = dict[key]
    #print mergeDict
    #print sorted(mergeDict, key = lambda x: mergeDict[x], reverse = True)
    for key in sorted(mergeDict, key = lambda x: mergeDict[x], reverse = True):
        #print key, mergeDict[key]
        result.append(key)
    return result

max_pair = None
max_score = 0
for i in range(0,20):
    print 'i', i
    for j in range(0,20):
        print 'j', j
        for k in range(0,20):
            print 'k', k
            for l in range(0,20):
                print 'l', l
                full_preds = [merge(tuple([weigh(exact_matches[p],i/10),weigh(preds[p], j/10),weigh(preds_m[p], k/10),weigh(most_common_clusters, l/10)]))[:5] for p in range(len(preds))]
                score = metrics.mapk([[l] for l in t2["hotel_cluster"]], full_preds, k=5)
                if max_score < score:
                    max_pair = (i,j,k,l)
                    max_score = score
print max_pair,max_score

i 0
j 0
k 0
l 0
l 1
l 2
l 3
l 4
l 5
l 6
l 7
l 8
l 9
l 10
l 11
l 12
l 13
l 14
l 15
l 16
l 17
l 18
l 19
k 1
l 0
l 1
l 2
l 3
l 4
l 5
l 6
l 7
l 8
l 9
l 10
l 11
l 12
l 13
l 14
l 15
l 16
l 17
l 18
l 19
k 2
l 0
l 1
l 2
l 3
l 4
l 5
l 6
l 7
l 8
l 9
l 10
l 11
l 12
l 13
l 14
l 15
l 16
l 17
l 18
l 19
k 3
l 0
l 1
l 2
l 3
l 4
l 5
l 6
l 7
l 8
l 9
l 10
l 11
l 12
l 13
l 14
l 15
l 16
l 17
l 18
l 19
k 4
l 0
l 1
l 2
l 3
l 4
l 5
l 6
l 7
l 8
l 9
l 10
l 11
l 12
l 13
l 14
l 15
l 16
l 17
l 18
l 19
k 5
l 0
l 1
l 2
l 3
l 4
l 5
l 6
l 7
l 8
l 9
l 10
l 11
l 12
l 13
l 14
l 15
l 16
l 17
l 18
l 19
k 6
l 0
l 1
l 2
l 3
l 4
l 5
l 6
l 7
l 8
l 9
l 10
l 11
l 12
l 13
l 14
l 15


KeyboardInterrupt: 